In [1]:
import pandas as pd
import torch
from model.FieldingModel import FieldingModel


/home/wahoo/.pyenv/versions/3.13.2/envs/ootp-analytics/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
league =  'BZ'
season_start = 2016
season_end = 20123
positions = [9,8,7,6,5,4,3,2]
ratings_type = "Standard"

In [3]:

    
for position in positions:
    print(f"\r\nTraining model for position {position}")
    fieldingModel = FieldingModel(league, season_start, season_end, position, ratings_type)
    fieldingModel.load_data(position)

    epoch_count, loss = fieldingModel.train(5000)
    print(f"Epoch [{epoch_count}/{5000}], Loss: {loss:.7f}")

    test_loss = fieldingModel.evaluate()
    print(f"Test Mean Squared Error: {test_loss:.7f}")

    feature_importance = fieldingModel.feature_importance()
    print(feature_importance)

    fieldingModel.save_model()


Training model for position 9
CUDA is available! Using GPU.
(52, 9)
(109, 9)
Empty DataFrame
Columns: [Age, WT, SPE, OFRngDelta, OFArmDelta, OFERRDelta, RF, ID, runsPAdjSeason]
Index: []
Epoch [2000/5000], Best Loss: 0.1529177
Epoch [4000/5000], Best Loss: 0.1085925
Epoch [4999/5000], Loss: 0.2334191
Test Absolute Error: 19.9020
Test Mean Squared Error: 19.9019547
   mean_abs_shap  stdev_abs_shap        name
3      12.502073       11.103165  OFRngDelta
6       6.882278        5.047034          RF
0       4.891610        4.508994         Age
4       4.278010        4.596299  OFArmDelta
5       4.250097        5.191754  OFERRDelta
1       3.701973        4.567683          WT
2       3.266198        2.854246         SPE
7       3.220996        2.109845          ID

Training model for position 8
CUDA is available! Using GPU.
(50, 8)
(104, 8)
Empty DataFrame
Columns: [Age, WT, SPE, OFRngDelta, OFArmDelta, OFERRDelta, CF, runsPAdjSeason]
Index: []
Epoch [2000/5000], Best Loss: 0.6744270
Epo